## Intro

Proficient Python programmers
use built-in data structures unless (or until) there is an obvious need to define a class.
There is no reason to add an extra level of abstraction if it doesn't help organize
our code. On the other hand, the "obvious" need is not always self-evident.

>Don't rush to use an object just because you can use an object, but
never neglect to create a class when you need to use a class.

## Adding behavior to class

>Python is very good at blurring
distinctions; it doesn't exactly help us to "think outside the box". Rather, it teaches
us to stop thinking about the box.

Many object-oriented languages (Java is the most notorious) teach us to never access
attributes directly. They insist that we write attribute access like this:

In [2]:
class Color:
    def __init__(self, rgb_value, name):
        self._rgb_value = rgb_value
        self._name = name

    def set_name(self, name):
        self._name = name
        
    def get_name(self):
        return self._name

The variables are prefixed with an underscore to suggest that they are *private* (other
languages would actually force them to be private). Then the get and set methods
provide access to each variable.

In [4]:
c = Color("#ff0000", "bright red")
print(c.get_name())
c.set_name("Red")
print(c.get_name())

bright red
Red


While in python, we usually do:

In [6]:
class Color:
    def __init__(self, rgb_value, name):
        self.rgb_value = rgb_value
        self.name = name
c = Color("#ff0000", "bright red")
print(c.name)
c.name = "red"

bright red


The reasoning is that we don't want to expose the private variables to the outside world. Plus, someday we may want to add extra code when a value is set or retrieved.
For example, we could decide to cache a value and return the cached value, or
we might want to validate that the value is a suitable input.<br>
 This doesn't make
much sense in Python since there isn't any real concept of private members!

However, Python gives us the `property` keyword to make methods look like attributes.

In [7]:
class Color:
    def __init__(self, rgb_value, name):
        self.rgb_value = rgb_value
        self._name = name

    def _set_name(self, name):
        if not name:
            raise Exception("Invalid Name")
        self._name = name

    def _get_name(self):
        return self._name
    name = property(_get_name, _set_name)

We have the `property` declaration at the bottom. It creates
a new attribute on the `Color` class called `name`, which now replaces the previous `name`
attribute. It sets this attribute to be a property, which calls the two methods we just
created whenever the property is accessed or changed. This new version of the `Color`
class can be used exactly the same way as the previous version, yet it now
does validation when we set the name attribute:

In [9]:
C = Color("#ff0000", "bright red")
print(C.name)
C.name = "red"
print(C.name)
C.name = ""

bright red
red


Exception: Invalid Name

Even with the name property, the previous code is not 100 percent
safe. People can still access the `_name` attribute directly and set it to an empty
string if they want to. *But if they access a variable we've explicitly marked with
an underscore to suggest it is private, they're the ones that have to deal with the
consequences, not us.*

### Properties in detail

This `property` constructor can actually accept two additional arguments, a deletion
function and a docstring for the property. The `delete` function is rarely supplied in
practice, but it can be useful for logging that a value has been deleted, or possibly
to veto deleting if we have reason to do so.

In [10]:
class Silly:
    def _get_silly(self):
        print("You are getting silly")
        return self._silly

    def _set_silly(self, value):
        print("You are making silly {}".format(value))
        self._silly = value

    def _del_silly(self):
        print("Whoah, you killed silly!")
        del self._silly
    silly = property(_get_silly, _set_silly,
    _del_silly, "This is a silly property")

In [12]:
s = Silly()
s.silly = "yes"
s.silly

You are making silly yes
You are getting silly


'yes'

In [13]:
del s.silly

Whoah, you killed silly!


### Create Property Using Decorator

The property function can be used with the decorator syntax to turn a get function
into a property:

In [18]:
class Foo:
    @property
    def foo(self):
        return "bar"
f=Foo()
f.foo

'bar'

This applies the property function as a decorator, and is equivalent to the previous
`foo = property(foo)` syntax. It means we don't have
to create private methods with underscore prefixes just to define a property.
<br>
Going one step further, we can specify a setter function for the new property.
First, we decorate the
`foo` method as a getter. Then, we decorate a second method with exactly the same
name by applying the setter attribute of the originally decorated `foo` method! The
property function returns an object; this object always comes with its own setter
attribute, which can then be applied as a decorator to other functions. Using the same
name for the get and set methods is not required, but it does help group the multiple
methods that access one property together.
We can also specify a deletion function with `@foo.deleter`.

Here's our previous `Silly` class rewritten to use `property` as a decorator:

In [19]:

class Silly:
    @property
    def silly(self):
        "This is a silly property"
        print("You are getting silly")
        return self._silly

    @silly.setter
    def silly(self, value):
        print("You are making silly {}".format(value))
        self._silly = value

    @silly.deleter
    def silly(self):
        print("Whoah, you killed silly!")
        del self._silly

s = Silly()
s.silly = "yes"
s.silly
del s.silly

You are making silly yes
You are getting silly
Whoah, you killed silly!


## Deciding when to use properties

Technically, in Python, data, properties, and methods are all attributes on a class.
The fact that a method is callable does not distinguish it from other types of
attributes; indeed it is possible to create normal objects that can be called like functions. We'll also
discover that functions and methods are themselves normal objects.<br>
The fact that methods are just callable attributes, and properties are just customizable
attributes can help us make this decision. Methods should typically represent actions;
things that can be done to, or performed by, the object. When you call a method, even
with only one argument, it should do something. *Method names are generally verbs.*

>A common need for custom behavior is caching
a value that is difficult to calculate or expensive to look up (requiring, for example,
a network request or database query). The goal is to store the value locally to avoid
repeated calls to the expensive calculation.

In [25]:
from urllib.request import urlopen
class WebPage:
    def __init__(self, url):
        self.url = url
        self._content = None

    @property
    def content(self):
        if not self._content:
            print("Retrieving New Page...")
            self._content = urlopen(self.url).read()
        return self._content

In [31]:
import time
webpage = WebPage("http://google.com/")
content1 = webpage.content
time.sleep(1)
content2 = webpage.content
content1 == content2

Retrieving New Page...


True

Custom getters are also useful for attributes that need to be calculated on the fly,
based on other object attributes.

In [32]:
class AverageList(list):
    @property
    def average(self):
        return sum(self) / len(self)

a = AverageList([1,2,3,4])
a.average

2.5

## Manager objects

Management objects are more
like office managers; they don't do the actual "visible" work out on the floor, but
without them, there would be no communication between departments and nobody
would know what they are supposed to do (although, this can be true anyway if the
organization is badly managed!). Analogously, the attributes on a management class
tend to refer to other objects that do the "visible" work; the behaviors on such a class
delegate to those other classes at the right time, and pass messages between them.

As an example, we'll write a program that does a find and replace action for text files
stored in a compressed ZIP file.
1. Unzipping the compressed file.
2. Performing the find and replace action.
3. Zipping up the new files.

In [35]:
import sys
import shutil
import zipfile
from pathlib import Path


class ZipReplace:
    def __init__(self, filename, search_string, replace_string):
        self.filename = filename
        self.search_string = search_string
        self.replace_string = replace_string
        self.temp_directory = Path("unzipped-{}".format(
        filename))

    def zip_find_replace(self):
        self.unzip_files()
        self.find_replace()
        self.zip_files()

    def unzip_files(self):
        self.temp_directory.mkdir()
        with zipfile.ZipFile(self.filename) as zip:
            zip.extractall(str(self.temp_directory))

    def find_replace(self):
        for filename in self.temp_directory.iterdir():
            with filename.open() as file:
                contents = file.read()
            contents = contents.replace(self.search_string, self.replace_string)
            with filename.open("w") as file:
                file.write(contents)

    def zip_files(self):
        with zipfile.ZipFile(self.filename, 'w') as file:
            for filename in self.temp_directory.iterdir():
                file.write(str(filename), filename.name)
        shutil.rmtree(str(self.temp_directory))
                

# if __name__ == "__main__":
#     ZipReplace(*sys.argv[1:4]).zip_find_replace()

The `zip_find_replace` function is a management object that coordinates the unzipping, searching, replacing, and zipping up of the files.

## Removing duplicate code

When we're trying to read two similar pieces of code, we have to
understand why they're different, as well as how they're different. This wastes the
reader's time; code should always be written to be readable first.

>The author once had to try to understand someone's code that had three identical
copies of the same 300 lines of very poorly written code. He had been
working with the code for a month before he finally comprehended that
the three "identical" versions were actually performing slightly different
tax calculations. Some of the subtle differences were intentional, but
there were also obvious areas where someone had updated a calculation
in one function without updating the other two. The number of subtle,
incomprehensible bugs in the code could not be counted. He eventually
replaced all 900 lines with an easy-to-read function of 20 lines or so.

>**In short: always make the effort to
refactor your code to be easier to read instead of writing bad code that is only easier
to write.**

We can see from the above example that we have repeated some part of our code.Let's explore two ways we can reuse existing code. After writing our code to replace
strings in a ZIP file full of text files, we are later contracted to scale all the images in
a ZIP file to 640 x 480. Looks like we could use a very similar paradigm to what we
used in ZipReplace. The first impulse might be to save a copy of that file and change
the find_replace method to scale_image or something similar. But this is not that a great idea. Instead, we'll create a superclass `ZipProcessor` for processing generic
ZIP files and using inheritance, we can stop repeating code and reuse the same code for both.

In [40]:
import os
import shutil
import zipfile
from pathlib import Path


class ZipProcessor:
    def __init__(self, zipname):
        self.zipname = zipname
        self.temp_directory = Path("unzipped-{}".format(zipname[:-4])) 

        def process_zip(self):
            self.unzip_files()
            self.process_files()
            self.zip_files()

        def unzip_files(self):
            self.temp_directory.mkdir()
            with zipfile.ZipFile(self.zipname) as zip:
                zip.extractall(str(self.temp_directory))

        def zip_files(self):
            with zipfile.ZipFile(self.zipname, 'w') as file:
                for filename in self.temp_directory.iterdir():
                    file.write(str(filename), filename.name)
            shutil.rmtree(str(self.temp_directory))

Now, let's fix up our original
`zipsearch` class to make use of this parent class:

In [42]:
# from zip_processor import ZipProcessor
import sys
import os


class ZipReplace(ZipProcessor):
    def __init__(self, filename, search_string,
    replace_string):
        super().__init__(filename)
        self.search_string = search_string
        self.replace_string = replace_string

    def process_files(self):
        '''perform a search and replace on all files in the
        temporary directory'''
        for filename in self.temp_directory.iterdir():
            with filename.open() as file:
                contents = file.read()
            contents = contents.replace(self.search_string, self.replace_string)
            with filename.open("w") as file:
                file.write(contents)

Using inheritance, it is now simple to create a photo scaling class that takes advantage of the
`ZipProcessor` functionality.

In [43]:
# from zip_processor import ZipProcessor
import sys
from PIL import Image


class ScaleZip(ZipProcessor):
    def process_files(self):
        '''Scale each image in the directory to 640x480'''
        for filename in self.temp_directory.iterdir():
            im = Image.open(str(filename))
            scaled = im.resize((640, 480))
            scaled.save(str(filename))

** We're Done!**

## Case study

This time, we'll be modeling a Document class that might be
used in a text editor or word processor. 

In [1]:
cd Case Study Ch.5

c:\Users\harik\Desktop\Python\Notes\OOP\Case Study Ch.5


In [3]:
from document import Document, Character
d = Document()
d.insert('h')
d.insert('e')
d.insert(Character('l', bold=True))
d.insert(Character('l', bold=True))
d.insert('o')
d.insert('\n')
d.insert(Character('w', italic=True))
d.insert(Character('o', italic=True))
d.insert(Character('r', underline=True))
d.insert('l')
d.insert('d')
print(d.string)

he*l*lo
/w/o_rld


In [4]:
d.cursor.home()
d.delete()
d.insert('W')
print(d.string)

he*l*lo
W/o_rld
